In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [13]:
# Read sensor 17 data
df_17 = pd.read_excel('data/sensor_17_combined_data.xlsx')
print("Sensor 17 data shape:", df_17.shape)
print("\nSensor 17 columns:", df_17.columns.tolist())
print("\nSensor 17 first few rows:")
df_17.head()


Sensor 17 data shape: (11561, 6)

Sensor 17 columns: ['datetime', 'upper_humidity', 'upper_temp_F', 'upper_irradiance_W_per_m2', 'sensor_id', 'power_W']

Sensor 17 first few rows:


,datetime,upper_humidity,upper_temp_F,upper_irradiance_W_per_m2,sensor_id,power_W
0,2025-08-12 17:10:00,50.11,93.23,178.20,17,6508.33
1,2025-08-12 17:20:00,49.13,92.92,170.52,17,5923.43
2,2025-08-12 17:30:00,52.38,91.76,103.87,17,927.46
3,2025-08-12 17:40:00,58.31,89.00,114.92,17,937.62
4,2025-08-12 17:50:00,62.02,87.62,129.81,17,4106.73


In [14]:
# Read sensor 20 data
df_20 = pd.read_excel('data/sensor_20_combined_data.xlsx')
print("Sensor 20 data shape:", df_20.shape)
print("\nSensor 20 columns:", df_20.columns.tolist())
print("\nSensor 20 first few rows:")
df_20.head()


Sensor 20 data shape: (10553, 6)

Sensor 20 columns: ['datetime', 'upper_humidity', 'upper_temp_F', 'upper_irradiance_W_per_m2', 'sensor_id', 'power_W']

Sensor 20 first few rows:


,datetime,upper_humidity,upper_temp_F,upper_irradiance_W_per_m2,sensor_id,power_W
0,2025-08-19 17:10:00,56.48,92.07,154.13,20,5280.41
1,2025-08-19 17:20:00,55.82,91.24,120.27,20,1209.58
2,2025-08-19 17:30:00,58.55,90.82,158.51,20,6394.42
3,2025-08-19 17:40:00,55.99,91.27,142.65,20,6228.92
4,2025-08-19 17:50:00,54.91,91.60,122.73,20,5970.66


In [18]:
def fit_lagged_model(file1_path, file2_path, sensor1_id=None, sensor2_id=None, 
                     treatment_sensor_id=None, maxlags=144, verbose=True):
    """
    Fit a lagged dependent variable model with HAC standard errors for two sensor files.
    
    Parameters:
    -----------
    file1_path : str
        Path to the first sensor Excel file
    file2_path : str
        Path to the second sensor Excel file
    sensor1_id : int, optional
        Sensor ID for the first file (if None, will use sensor_id from the data)
    sensor2_id : int, optional
        Sensor ID for the second file (if None, will use sensor_id from the data)
    treatment_sensor_id : int, optional
        Sensor ID to use as treatment group (1). Other sensor will be control (0).
        If None, uses sensor2_id as treatment.
    maxlags : int, default=144
        Maximum lags for HAC standard errors
    verbose : bool, default=True
        Whether to print model summary
        
    Returns:
    --------
    dict : Dictionary containing:
        - 'result': OLS regression results object
        - 'df_lagged': DataFrame with lagged variables
        - 'df_combined': Combined DataFrame
    """
    # Load data
    df1 = pd.read_excel(file1_path)
    df2 = pd.read_excel(file2_path)
    
    # Ensure datetime is datetime type
    df1['datetime'] = pd.to_datetime(df1['datetime'])
    df2['datetime'] = pd.to_datetime(df2['datetime'])
    
    # Combine data
    df_combined = pd.concat([df1, df2], ignore_index=True)
    df_combined = df_combined.sort_values(['sensor_id', 'datetime']).reset_index(drop=True)
    
    # Determine treatment sensor ID
    if treatment_sensor_id is None:
        # Use the sensor_id from the second file as treatment
        if sensor2_id is None:
            treatment_sensor_id = df2['sensor_id'].iloc[0]
        else:
            treatment_sensor_id = sensor2_id
    
    # Create treatment variable (0 for control, 1 for treatment)
    df_combined['treatment'] = (df_combined['sensor_id'] == treatment_sensor_id).astype(int)
    
    # Create interaction terms
    df_combined['treat_irrad'] = df_combined['treatment'] * df_combined['upper_irradiance_W_per_m2']
    df_combined['treat_temp'] = df_combined['treatment'] * df_combined['upper_temp_F']
    df_combined['treat_humid'] = df_combined['treatment'] * df_combined['upper_humidity']
    
    # Create lagged power variable (grouped by sensor_id)
    df_combined['power_W_lag1'] = df_combined.groupby('sensor_id')['power_W'].shift(1)
    
    # Remove rows with missing lagged values (first observation of each sensor)
    df_lagged = df_combined.dropna(subset=['power_W_lag1']).reset_index(drop=True)
    
    # Prepare fixed effects (X) and dependent variable (y)
    X_lag = sm.add_constant(df_lagged[[
        'treatment',
        'upper_irradiance_W_per_m2',
        'upper_temp_F',
        'upper_humidity',
        'treat_irrad',
        'treat_temp',
        'treat_humid',
        'power_W_lag1'  # Lagged dependent variable
    ]])
    
    y_lag = df_lagged['power_W']
    
    # Fit OLS model with HAC standard errors
    model_lag = sm.OLS(y_lag, X_lag)
    result_lag = model_lag.fit(cov_type='HAC', cov_kwds={'maxlags': maxlags})
    
    if verbose:
        print("=" * 80)
        print("LAGGED DEPENDENT VARIABLE MODEL WITH HAC STANDARD ERRORS")
        print("=" * 80)
        print(f"File 1: {file1_path}")
        print(f"File 2: {file2_path}")
        print(f"Treatment sensor ID: {treatment_sensor_id}")
        print(f"Combined data shape: {df_combined.shape}")
        print(f"Lagged data shape: {df_lagged.shape}")
        print("=" * 80)
        print(result_lag.summary())
    
    return {
        'result': result_lag,
        'df_lagged': df_lagged,
        'df_combined': df_combined,
        'treatment_sensor_id': treatment_sensor_id
    }



# Lagged Dependent Variable Model with HAC Standard Errors

To address autocorrelation, we include lagged values of the dependent variable (power) as predictors:

Power_ij = β₀ + β₁·Treatment_i + β₂·Irradiance_ij + β₃·Temperature_ij + β₄·Humidity_ij 
         + β₅·(Treatment × Irradiance)_ij + β₆·(Treatment × Temperature)_ij 
         + β₇·(Treatment × Humidity)_ij + β₈·Power_{i,j-1} + ε_ij

Where:
- i = sensor/panel (treatment indicator)
- j = observation within sensor
- Power_{i,j-1} = lagged power (previous observation)
- ε_ij = error term

**Key Features:**
- Fixed effects only (no random effects) - avoids requirement of N > 2 groups
- Lagged dependent variable captures persistence in power output over time
- HAC (Heteroskedasticity and Autocorrelation Consistent) standard errors to account for autocorrelation
- Interaction terms test whether relationships differ by treatment


## Usage Example

You can now use the `fit_lagged_model()` function with any two sensor files:

```python
# Example 1: Using sensor 17 and sensor 20 (default - sensor 20 as treatment)
results = fit_lagged_model(
    'data/sensor_17_combined_data.xlsx',
    'data/sensor_20_combined_data.xlsx'
)

# Example 2: Specify which sensor is treatment
results = fit_lagged_model(
    'data/sensor_17_combined_data.xlsx',
    'data/sensor_20_combined_data.xlsx',
    treatment_sensor_id=20  # Sensor 20 is treatment
)

# Example 3: Use sensor 17 as treatment instead
results = fit_lagged_model(
    'data/sensor_17_combined_data.xlsx',
    'data/sensor_20_combined_data.xlsx',
    treatment_sensor_id=17  # Sensor 17 is treatment
)

# Example 4: Use different files
results = fit_lagged_model(
    'data/sensor_1_combined_data.xlsx',
    'data/sensor_17_combined_data.xlsx'
)

# Access results
model_result = results['result']  # OLS regression results
df_lagged = results['df_lagged']  # DataFrame with lagged variables
df_combined = results['df_combined']  # Combined DataFrame
```



In [22]:
# Example: Run model with sensor 17 and sensor 20
results_17_20 = fit_lagged_model(
    'data/sensor_17_combined_data.xlsx',
    'data/sensor_24_combined_data.xlsx',
    treatment_sensor_id=24  
)

# Access the results
model_result = results_17_20['result']
df_lagged = results_17_20['df_lagged']
df_combined = results_17_20['df_combined']



LAGGED DEPENDENT VARIABLE MODEL WITH HAC STANDARD ERRORS
File 1: data/sensor_17_combined_data.xlsx
File 2: data/sensor_24_combined_data.xlsx
Treatment sensor ID: 24
Combined data shape: (19107, 11)
Lagged data shape: (19105, 11)
                            OLS Regression Results                            
Dep. Variable:                power_W   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                 1.559e+04
Date:                Tue, 13 Jan 2026   Prob (F-statistic):               0.00
Time:                        16:09:06   Log-Likelihood:            -1.5442e+05
No. Observations:               19105   AIC:                         3.089e+05
Df Residuals:                   19096   BIC:                         3.089e+05
Df Model:                           8                                         
Covariance Type:                  HAC                       